In [ ]:
import random
import time
from enum import Enum, auto
from typing import List, Dict, Tuple, Optional, Set

In [ ]:
# Перечисление для времени суток
class TimeOfDay(Enum):
    MORNING = auto()  # Утро
    DAY = auto()      # День
    EVENING = auto()  # Вечер
    NIGHT = auto()    # Ночь

# Класс, представляющий мир - основное окружение для симуляции
class World:
    def __init__(self, width: int, height: int):
        self.width = width    # Ширина мира в клетках
        self.height = height  # Высота мира в клетках
        # Сетка мира (двумерный список). None - пустая клетка
        self.grid = [[None for _ in range(width)] for _ in range(height)]
        self.time_of_day = TimeOfDay.MORNING  # Текущее время суток
        self.time_ticks = 0    # Счетчик шагов симуляции
        self.entities = []     # Список всех существ в мире
        self.day_counter = 0   # Счетчик дней

    # Добавление существа в указанные координаты
    def add_entity(self, entity, x: int, y: int):
        # Проверка выхода за границы мира
        if not (0 <= x < self.width and 0 <= y < self.height):
            return False

        # Проверка, что клетка пуста
        if self.grid[y][x] is not None:
            return False

        # Установка координат существа и ссылки на мир
        entity.x = x
        entity.y = y
        entity.world = self
        # Размещение в сетке и добавление в общий список
        self.grid[y][x] = entity
        self.entities.append(entity)
        return True

    # Удаление существа из мира
    def remove_entity(self, entity):
        if entity in self.entities:
            self.entities.remove(entity)
        # Очистка клетки, если в ней находится это существо
        if (0 <= entity.x < self.width and
            0 <= entity.y < self.height and
            self.grid[entity.y][entity.x] == entity):
            self.grid[entity.y][entity.x] = None

    # Получение списка соседних клеток (включая диагонали)
    def get_neighbors(self, x: int, y: int) -> List[Tuple[int, int]]:
        neighbors = []
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                if dx == 0 and dy == 0:  # Исключаем текущую клетку
                    continue
                nx, ny = x + dx, y + dy
                # Проверка, что координаты в пределах мира
                if 0 <= nx < self.width and 0 <= ny < self.height:
                    neighbors.append((nx, ny))
        return neighbors

    # Обновление времени в мире
    def update_time(self):
        self.time_ticks += 1
        # Каждые 6 шагов меняется время суток
        if self.time_ticks % 6 == 0:
            prev_time = self.time_of_day
            # Циклическое изменение времени суток: Утро -> День -> Вечер -> Ночь -> Утро...
            self.time_of_day = {
                TimeOfDay.MORNING: TimeOfDay.DAY,
                TimeOfDay.DAY: TimeOfDay.EVENING,
                TimeOfDay.EVENING: TimeOfDay.NIGHT,
                TimeOfDay.NIGHT: TimeOfDay.MORNING
            }[self.time_of_day]
            
            # Увеличиваем счетчик дней при переходе от ночи к утру
            if prev_time == TimeOfDay.NIGHT and self.time_of_day == TimeOfDay.MORNING:
                self.day_counter += 1

    # Один шаг симуляции
    def step(self):
        self.update_time()  # Сначала обновляем время

        # Обрабатываем растения
        for entity in self.entities[:]:  # Копия списка для безопасного изменения
            if isinstance(entity, Plant):
                entity.grow()  # Растения растут
        
        # Обрабатываем животных в случайном порядке
        for entity in random.sample(self.entities[:], len(self.entities[:])):
            if isinstance(entity, Animal):
                entity.update_behavior()  # Обновление поведения
                entity.move()            # Движение
                entity.eat()             # Питание
                entity.reproduce()       # Размножение
                entity.update_hunger()   # Обновление уровня голода

                # Удаление, если энергия исчерпана
                if entity.energy <= 0:
                    self.remove_entity(entity)

    # Отображение текущего состояния мира в консоли
    def display(self):
        # Соответствие времени суток и его названия
        time_names = {
            TimeOfDay.MORNING: "Утро",
            TimeOfDay.DAY: "День",
            TimeOfDay.EVENING: "Вечер",
            TimeOfDay.NIGHT: "Ночь"
        }
        print(f"День: {self.day_counter} Время: {time_names[self.time_of_day]} (Шаг: {self.time_ticks})")
        print(f"Существ: {len(self.entities)} (Растений: {sum(1 for e in self.entities if isinstance(e, Plant))}, "
              f"Травоядных: {sum(1 for e in self.entities if isinstance(e, Pauvre))}, "
              f"Хищников: {sum(1 for e in self.entities if isinstance(e, Malheureux))})")

        # Отображение сетки
        for y in range(self.height):
            row = []
            for x in range(self.width):
                entity = self.grid[y][x]
                if entity is None:
                    row.append('.')  # Пустая клетка
                elif isinstance(entity, Lumiere):
                    # 'L' - активно, 'l' - неактивно
                    row.append('L' if entity.active else 'l')
                elif isinstance(entity, Obscurite):
                    row.append('O' if entity.active else 'o')
                elif isinstance(entity, Demi):
                    row.append('D' if entity.active else 'd')
                elif isinstance(entity, Pauvre):
                    # 'P' - бодрствует, 'p' - спит
                    row.append('P' if not entity.sleeping else 'p')
                elif isinstance(entity, Malheureux):
                    row.append('M' if not entity.sleeping else 'm')
            print(' '.join(row))
        print()

# Базовый класс для растений
class Plant:
    def __init__(self):
        self.x = 0          # Координата X
        self.y = 0          # Координата Y
        self.world = None   # Ссылка на мир
        self.growth_rate = 1  # Скорость роста
        self.active = False  # Активен ли рост
        self.health = 100    # Здоровье растения

    # Проверка, может ли растение расти в текущее время суток
    def can_grow_in_time(self, time_of_day: TimeOfDay) -> bool:
        raise NotImplementedError  # Абстрактный метод

    # Процесс роста растения
    def grow(self):
        if self.world is None:
            return

        # Проверка активности в текущее время
        self.active = self.can_grow_in_time(self.world.time_of_day)
        
        if not self.active:
            return

        # Восстановление здоровья при активности
        self.health = min(100, self.health + 5)
        
        # Получение соседей в случайном порядке
        neighbors = self.world.get_neighbors(self.x, self.y)
        random.shuffle(neighbors)
        
        # Попытка распространения на соседние клетки
        for nx, ny in neighbors:
            target = self.world.grid[ny][nx]
            
            # Если клетка пуста - попытка размножения
            if target is None and random.random() < 0.2:
                new_plant = self.__class__()
                if self.world.add_entity(new_plant, nx, ny):
                    continue
            # Если в клетке другое растение - попытка вытеснения
            elif (isinstance(target, Plant) and 
                  target != self and 
                  self.active and not target.active and 
                  random.random() < 0.3):
                target.health -= 20
                if target.health <= 0:
                    self.world.remove_entity(target)
                    new_plant = self.__class__()
                    self.world.add_entity(new_plant, nx, ny)

    def __str__(self):
        return f"{self.__class__.__name__} в ({self.x}, {self.y})"

# Класс растения Lumiere (растет днем)
class Lumiere(Plant):
    def can_grow_in_time(self, time_of_day: TimeOfDay) -> bool:
        return time_of_day == TimeOfDay.DAY

# Класс растения Obscurite (растет ночью)
class Obscurite(Plant):
    def can_grow_in_time(self, time_of_day: TimeOfDay) -> bool:
        return time_of_day == TimeOfDay.NIGHT

# Класс растения Demi (растет утром и вечером)
class Demi(Plant):
    def can_grow_in_time(self, time_of_day: TimeOfDay) -> bool:
        return time_of_day in (TimeOfDay.MORNING, TimeOfDay.EVENING)

# Базовый класс для животных
class Animal:
    def __init__(self):
        self.x = 0          # Координата X
        self.y = 0          # Координата Y
        self.world = None   # Ссылка на мир
        self.energy = 100   # Уровень энергии
        self.hunger = 0     # Уровень голода
        self.sleeping = False  # Спящий режим
        self.age = 0        # Возраст (в шагах симуляции)

    # Обновление поведения животного
    def update_behavior(self):
        if self.world is None:
            return
        self.age += 1
        self.modify_behavior()  # Вызов специфичного поведения

    # Абстрактный метод для изменения поведения
    def modify_behavior(self):
        raise NotImplementedError

    # Движение животного
    def move(self):
        if self.sleeping or self.world is None:
            return

        # Возможные клетки для движения (пустые или с растениями)
        possible_moves = [pos for pos in self.world.get_neighbors(self.x, self.y)
                         if self.world.grid[pos[1]][pos[0]] is None or 
                         (isinstance(self.world.grid[pos[1]][pos[0]], Plant) and 
                         random.random() < 0.3)]
        
        # Перемещение с учетом скорости
        if possible_moves and random.random() < self.get_speed():
            nx, ny = random.choice(possible_moves)
            self.world.grid[self.y][self.x] = None
            self.x, self.y = nx, ny
            self.world.grid[ny][nx] = self
            self.energy -= 1  # Затраты энергии на движение

    # Расчет скорости движения (зависит от голода)
    def get_speed(self) -> float:
        return max(0.1, 1.0 - (self.hunger / 200))

    # Процесс питания
    def eat(self):
        if self.sleeping or self.hunger < 20 or self.world is None:
            return

        # Поиск пищи среди соседей
        for nx, ny in self.world.get_neighbors(self.x, self.y):
            target = self.world.grid[ny][nx]
            if target is None:
                continue
                
            # Проверка, является ли цель пищей
            if any(isinstance(target, food_type) for food_type in self.get_food_types()):
                nutrition = self.get_nutrition(target)
                self.world.remove_entity(target)
                self.hunger = max(0, self.hunger - nutrition)
                self.energy += nutrition / 2
                return

    # Абстрактный метод для получения типов пищи
    def get_food_types(self) -> Tuple:
        raise NotImplementedError

    # Получение питательной ценности пищи
    def get_nutrition(self, food) -> int:
        if isinstance(food, Plant):
            return 30
        elif isinstance(food, Animal):
            return 50
        return 0

    # Обновление уровня голода
    def update_hunger(self):
        self.hunger += 2
        if self.hunger > 100:  # Голод снижает энергию
            self.energy -= 5

    # Процесс размножения
    def reproduce(self):
        # Условия для размножения: достаточно энергии, не голоден, случайный шанс
        if (self.energy < 80 or 
            self.hunger > 50 or 
            random.random() > 0.05 or 
            self.world is None):
            return

        # Поиск партнера для размножения среди соседей
        for nx, ny in self.world.get_neighbors(self.x, self.y):
            target = self.world.grid[ny][nx]
            if (target is not None and 
                isinstance(target, self.__class__) and 
                target != self and 
                target.energy > 70 and 
                target.hunger < 50):
                
                # Поиск пустой клетки для потомства
                for cx, cy in self.world.get_neighbors(self.x, self.y):
                    if self.world.grid[cy][cx] is None:
                        new_animal = self.__class__()
                        new_animal.energy = 60
                        if self.world.add_entity(new_animal, cx, cy):
                            self.energy -= 30
                            target.energy -= 30
                            return

    def __str__(self):
        return f"{self.__class__.__name__} в ({self.x}, {self.y})"

# Класс травоядного животного Pauvre
class Pauvre(Animal):
    def __init__(self):
        super().__init__()
        self.aggression = 0    # Уровень агрессии
        self.group = set()     # Группа особей
        self.group_size = 1    # Размер группы

    # Изменение специфичного поведения
    def modify_behavior(self):
        # Сон ночью
        self.sleeping = (self.world.time_of_day == TimeOfDay.NIGHT)
        
        # Агрессия зависит от голода и размера группы
        self.aggression = min(100, self.hunger * 0.8 + len(self.group) * 5)
        
        # Обновление состава группы
        self.update_group()

    # Обновление группы особей
    def update_group(self):
        self.group = {self}
        # Поиск особей в радиусе 3 клеток
        for dx in range(-3, 4):
            for dy in range(-3, 4):
                nx, ny = self.x + dx, self.y + dy
                if (0 <= nx < self.world.width and 
                    0 <= ny < self.world.height and 
                    isinstance(self.world.grid[ny][nx], Pauvre)):
                    self.group.add(self.world.grid[ny][nx])
        
        self.group_size = len(self.group)
        
        # Разделение группы при перенаселении
        if self.group_size > 5 and random.random() < 0.1:
            self.split_group()

    # Разделение группы на две подгруппы
    def split_group(self):
        # Вычисление центра масс группы
        avg_x = sum(m.x for m in self.group) / len(self.group)
        avg_y = sum(m.y for m in self.group) / len(self.group)
        
        # Разделение на две группы относительно центра
        subgroup1 = set()
        subgroup2 = set()
        
        for member in self.group:
            if member.x < avg_x or (member.x == avg_x and member.y < avg_y):
                subgroup1.add(member)
            else:
                subgroup2.add(member)
        
        # Попытка разойтись в пространстве
        for member in subgroup1:
            # Движение влево или вверх
            dx = -1 if random.random() < 0.5 else 0
            dy = -1 if dx == 0 else 0
            nx, ny = member.x + dx, member.y + dy
            if (0 <= nx < self.world.width and 
                0 <= ny < self.world.height and 
                self.world.grid[ny][nx] is None):
                self.world.grid[member.y][member.x] = None
                member.x, member.y = nx, ny
                self.world.grid[ny][nx] = member

    # Типы пищи (только Lumiere)
    def get_food_types(self) -> Tuple:
        return (Lumiere,)

    # Питательная ценность пищи (зависит от времени суток)
    def get_nutrition(self, food) -> int:
        if isinstance(food, Lumiere):
            # Больше питательных веществ утром
            if self.world.time_of_day == TimeOfDay.MORNING:
                return 50
            elif self.world.time_of_day == TimeOfDay.EVENING:
                return 20
            return 30
        return 0

    # Процесс питания с учетом агрессии
    def eat(self):
        if self.sleeping or self.hunger < 20 or self.world is None:
            return

        # Сначала пробуем съесть растения
        super().eat()
        
        # Если голодны и агрессивны - атакуем других травоядных не из своей группы
        if self.hunger > 50 and self.aggression > 60:
            for nx, ny in self.world.get_neighbors(self.x, self.y):
                target = self.world.grid[ny][nx]
                if (isinstance(target, Pauvre) and 
                    target not in self.group and 
                    random.random() < 0.3):
                    
                    damage = min(20, self.aggression / 3)
                    target.energy -= damage
                    self.energy -= 5
                    self.hunger = max(0, self.hunger - 10)
                    return

# Класс хищника Malheureux
class Malheureux(Animal):
    def __init__(self):
        super().__init__()
        self.pack = set()    # Стая
        self.hunting = False # Режим охоты

    # Изменение специфичного поведения
    def modify_behavior(self):
        # Активны на рассвете и закате
        self.sleeping = (self.world.time_of_day in (TimeOfDay.DAY, TimeOfDay.NIGHT))
        
        # Охотничий режим при голоде и активности
        self.hunting = (not self.sleeping and self.hunger > 40)
        
        # Обновление состава стаи
        self.update_pack()

    # Обновление стаи
    def update_pack(self):
        self.pack = {self}
        # Поиск особей в радиусе 2 клеток
        for dx in range(-2, 3):
            for dy in range(-2, 3):
                nx, ny = self.x + dx, self.y + dy
                if (0 <= nx < self.world.width and 
                    0 <= ny < self.world.height and 
                    isinstance(self.world.grid[ny][nx], Malheureux)):
                    self.pack.add(self.world.grid[ny][nx])
        
        # Повышение агрессии при большом размере стаи
        if len(self.pack) > 3:
            for member in self.pack:
                member.aggression = True

    # Типы пищи (растения Demi, Obscurite и травоядные Pauvre)
    def get_food_types(self) -> Tuple:
        return (Demi, Obscurite, Pauvre)

    # Скорость движения с учетом режима охоты
    def get_speed(self) -> float:
        base_speed = super().get_speed()
        if self.hunting:
            return base_speed * 1.2  # Быстрее при охоте
        return base_speed * 0.8      # Медленнее в обычном режиме

    # Процесс питания с учетом охоты
    def eat(self):
        if self.sleeping or self.hunger < 20 or self.world is None:
            return

        # Сначала пробуем съесть растения
        super().eat()
        
        # Если в режиме охоты и голодны - атакуем травоядных
        if self.hunting and self.hunger > 40:
            for nx, ny in self.world.get_neighbors(self.x, self.y):
                target = self.world.grid[ny][nx]
                if isinstance(target, Pauvre) and random.random() < 0.4:
                    # Бонус за размер стаи
                    pack_bonus = 1.0 + (len(self.pack) * 0.2)
                    damage = min(30, 15 * pack_bonus)
                    target.energy -= damage
                    self.energy -= 5
                    self.hunger = max(0, self.hunger - 25)
                    
                    # Удаление жертвы, если погибла
                    if target.energy <= 0:
                        self.world.remove_entity(target)
                    return

# Инициализация мира с существами
def initialize_world(world: World, plant_density: float = 0.2, 
                    pauvre_count: int = 10, malheureux_count: int = 5):
    plant_classes = [Lumiere, Obscurite, Demi]  # Доступные типы растений
    
    # Расчет количества растений на основе плотности
    num_plants = int(world.width * world.height * plant_density)
    
    # Добавление растений
    for _ in range(num_plants):
        while True:
            x, y = random.randint(0, world.width-1), random.randint(0, world.height-1)
            if world.grid[y][x] is None:
                plant_class = random.choice(plant_classes)
                plant = plant_class()
                if world.add_entity(plant, x, y):
                    break
    
    # Добавление травоядных
    for _ in range(pauvre_count):
        while True:
            x, y = random.randint(0, world.width-1), random.randint(0, world.height-1)
            if world.grid[y][x] is None:
                pauvre = Pauvre()
                if world.add_entity(pauvre, x, y):
                    break
    
    # Добавление хищников
    for _ in range(malheureux_count):
        while True:
            x, y = random.randint(0, world.width-1), random.randint(0, world.height-1)
            if world.grid[y][x] is None:
                malheureux = Malheureux()
                if world.add_entity(malheureux, x, y):
                    break

# Запуск симуляции
def run_simulation(width: int = 20, height: int = 20, 
                   plant_density: float = 0.2, 
                   pauvre_count: int = 10, malheureux_count: int = 5,
                   steps: int = 100, delay: float = 0.5):
    # Создание мира
    world = World(width, height)
    # Инициализация существ
    initialize_world(world, plant_density, pauvre_count, malheureux_count)
    
    try:
        # Основной цикл симуляции
        for _ in range(steps):
            world.step()      # Шаг симуляции
            world.display()   # Отображение состояния
            time.sleep(delay) # Задержка для удобства наблюдения
    except KeyboardInterrupt:
        print("\nСимуляция остановлена пользователем.")
    except Exception as e:
        print(f"Ошибка: {e}")
    
    # Вывод итоговой статистики
    print("Симуляция завершена.")
    print(f"Итоговые показатели: День {world.day_counter}, Всего существ: {len(world.entities)}")
    print(f"Растений: {sum(1 for e in world.entities if isinstance(e, Plant))}")
    print(f"Травоядных: {sum(1 for e in world.entities if isinstance(e, Pauvre))}")
    print(f"Хищников: {sum(1 for e in world.entities if isinstance(e, Malheureux))}")

# Точка входа в программу
if __name__ == "__main__":
    # Параметры симуляции:
    run_simulation(
        width=20,            # Ширина мира
        height=20,           # Высота мира
        plant_density=0.2,   # Плотность растений (0.0 - 1.0)
        pauvre_count=15,     # Начальное количество травоядных
        malheureux_count=5,  # Начальное количество хищников
        steps=200,           # Количество шагов симуляции
        delay=0.3            # Задержка между шагами (сек)
    )

Day: 0 Time: Morning (Tick: 1)
Entities: 138 (Plants: 117, Pauvres: 16, Malheureux: 5)
. . . . . . . l o . . . . . . . P l P o
l . . . P . d o . . . . . o o . . D . o
. . . . . D . . . . . . . . . P D d P D
. . . . d d . . . . . M . . . d . o . D
. . . . . . l . . . . . . . . . . o d D
. l . . . d D o . . . . o l . . l . . l
. . . . . . . d D . . l d d l d . . . .
D . . . . . . . . . P . D . d D . . . .
. . . . . . D . . . . l . . . d . . . .
. . . o . . . d . d D l . . . l d d . .
. . . . . D . . D . d . o . . d D . P D
. P . . d P . . . . P . . P . o d . d .
D d . . o . . d M . . . . . P . . . . .
l . . . . . . D D . . l . l . . D o o d
. . . . . . D . . d . . . . . P d D D l
l . . . P . l . . . . P . M . . . d d .
. d D . . . o . o . . l . . . . . D . .
D d . d l . . . o D d . . . . l . l d .
. o . . . . . M . D . . o . . . . M . .
. . . . o o P . . d . . . . . . . . . .

Day: 0 Time: Morning (Tick: 2)
Entities: 178 (Plants: 157, Pauvres: 16, Malheureux: 5)
. . . . d P . l o . . . .